
# Predicting Evergreeness of Content
<img src="http://techvela.com/wp-content/uploads/2012/12/social-media-agency-leeds.png" width=500/>


We will be eventually using decision trees to classify certain aspects of [Stumbleupon](www.stumbleupon.com), a web page recommender. A description of the columns of the dataset is below:

FieldName|Type|Description
---------|----|-----------
url|string|Url of the webpage to be classified
title|string|Title of the article
body|string|Body text of article
urlid|integer| StumbleUpon's unique identifier for each url
boilerplate|json|Boilerplate text
alchemy_category|string|Alchemy category (per the publicly available Alchemy API found at www.alchemyapi.com)
alchemy_category_score|double|Alchemy category score (per the publicly available Alchemy API found at www.alchemyapi.com)
avglinksize| double|Average number of words in each link
commonlinkratio_1|double|# of links sharing at least 1 word with 1 other links / # of links
commonlinkratio_2|double|# of links sharing at least 1 word with 2 other links / # of links
commonlinkratio_3|double|# of links sharing at least 1 word with 3 other links / # of links
commonlinkratio_4|double|# of links sharing at least 1 word with 4 other links / # of links
compression_ratio|double|Compression achieved on this page via gzip (measure of redundancy)
embed_ratio|double|Count of number of <embed> usage
frameBased|integer (0 or 1)|A page is frame-based (1) if it has no body markup but have a frameset markup
frameTagRatio|double|Ratio of iframe markups over total number of markups
hasDomainLink|integer (0 or 1)|True (1) if it contains an <a> with an url with domain
html_ratio|double|Ratio of tags vs text in the page
image_ratio|double|Ratio of <img> tags vs text in the page
is_news|integer (0 or 1) | True (1) if StumbleUpon's news classifier determines that this webpage is news
lengthyLinkDomain| integer (0 or 1)|True (1) if at least 3 <a> 's text contains more than 30 alphanumeric characters
linkwordscore|double|Percentage of words on the page that are in hyperlink's text
news_front_page| integer (0 or 1)|True (1) if StumbleUpon's news classifier determines that this webpage is front-page news
non_markup_alphanum_characters|integer| Page's text's number of alphanumeric characters
numberOfLinks|integer Number of <a>|markups
numwords_in_url| double|Number of words in url
parametrizedLinkRatio|double|A link is parametrized if it's url contains parameters or has an attached onClick event
spelling_errors_ratio|double|Ratio of words not found in wiki (considered to be a spelling mistake)
label|integer (0 or 1)|User-determined label. Either evergreen (1) or non-evergreen (0); available for train.tsv only

In [ ]:
import pandas as pd
import json

pd.options.display.max_columns=30

In [ ]:
# Load up the data

data = pd.read_csv("stumbleupon.tsv", sep='\t')
data['title'] = data.boilerplate.map(lambda x: json.loads(x).get('title', ''))
data['body'] = data.boilerplate.map(lambda x: json.loads(x).get('body', ''))
data.head()

## Predicting "Greenness" Of Content

### What are 'evergreen' sites?

> #### Evergreen sites are those that are always relevant.  As opposed to breaking news or current events, evergreen websites are relevant no matter the time or season. 

> #### A sample of URLs is below, where label = 1 are 'evergreen' websites

In [ ]:
data[['url', 'label']].head()

# Exercise

 ### Exercise: 1. In a group: Brainstorm 3 - 5 features you could develop that would be useful for predicting evergreen websites.
 ###  Exercise: 2. After looking at the dataset, can you model or quantify any of the characteristics you wanted?
- I.E. If you believe high-image content websites are likely to be evergreen, how can you build a feature that represents that?
- I.E. If you believe weather content is likely NOT to be evergreen, how might you build a feature that represents that?

### Split up and develop 1-3 of the those features independently.

### Exercise: 3. Does being a news site affect evergreeness? 
Compute or plot the percentage of news related evergreen sites.

### Exercise: 4. Does category in general affect evergreeness? 
Plot the rate of evergreen sites for all Alchemy categories.

### Exercise: 5. How many articles are there per category?

### Exercise: 6. Create a feature for the title containing 'recipe'. 
Is the % of evegreen websites higher or lower on pages that have recipe in the the title?

#  Let's Explore Some Decision Trees

Let's use a decision tree to predict the "evergreenness" of a website. Not only will the decision tree give us class predictions, but we'll also get to see the features and rule-flow that led to that prediction.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

model = DecisionTreeClassifier()

X = data[['image_ratio', 'html_ratio', 'recipe', 'label']].dropna()
y = X['label']
X.drop('label', axis=1, inplace=True)


# Fits the model
model.fit(X, y)


from IPython.display import Image
from sklearn.externals.six import StringIO
import pydot

#dot_data = StringIO()
#tree.export_graphviz(model,out_file=dot_data)

# From the stringio, create a graph and output to screen
#graph = pydot.graph_from_dot_data(dot_data.getvalue())
#Image(graph.create_png())

We can use the 'feature\_importances\_' attribute to view more information about which features were most used. These are based on Gini impurity, and normalized amongst the factors.

In [ ]:
features = X.columns
feature_importances = model.feature_importances_

features_df = pd.DataFrame({'Features': features, 'Importance Score': feature_importances})
features_df.sort('Importance Score', inplace=True, ascending=False)

features_df.head()

# Exercise

Evaluate the decision tree using cross-validation; use AUC as the evaluation metric.

#  Adjusting Decision Trees to Avoid Overfitting

By default, sklearn trees defines how large to grow the tree (which is quite large). We can instead control for overfitting on our own. To do this, we can adjust the maximum number of questions (max_depth) or the minimum number of records in each final node (min_samples_leaf).

Read the [documentation](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier) for more information on settings you can adjust.

In [ ]:
from IPython.display import Image
from sklearn.externals.six import StringIO
import pydot

def build_tree_image(model,feat_names=None):
    dot_data = StringIO()
    tree.export_graphviz(model,out_file=dot_data,feature_names=feat_names)
    
    # From the stringio, create a graph and output to screen
    graph = pydot.graph_from_dot_data(dot_data.getvalue())
    return graph.create_png()

In [ ]:
model = DecisionTreeClassifier(
                max_depth = 2,
                min_samples_leaf = 5)

model.fit(X, y)
Image(build_tree_image(model,['image_ratio', 'html_ratio', 'recipe']))

In [ ]:

scores = cross_val_score(model, X, y, scoring='roc_auc', cv=5)
print('CV AUC {}, Average AUC {}'.format(scores, scores.mean()))

 ### Demo: Build a random forest model to predict the evergreeness of a website. 

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators = 20)

model.fit(X, y)

### Demo: Extracting importance of features

In [ ]:
features = X.columns
feature_importances = model.feature_importances_

features_df = pd.DataFrame({'Features': features, 'Importance Score': feature_importances})
features_df.sort('Importance Score', inplace=True, ascending=False)

features_df.head()

# Exercise

Evaluate the Random Forest model using cross-validation; increase the number of estimators (i.e. number of trees) and view how that improves predictive performance.

##  Independent Practice: Evaluate Random Forest Using Cross-Validation

1. Continue adding input variables to the model that you think may be relevant
2. For each feature:
  - Evaluate the model for improved predictive performance using cross-validation
  - Evaluate the _importance_ of the feature
  - 
3. **Bonus**: Just like the 'recipe' feature, add in similar text features and evaluate their performance.
